# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [99]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

url=  "https://ycharts.com/companies/TSLA/revenues"
#Headers es para que no salga error porque no sepan que eres una maquina 

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36' }
response = requests.get(url, headers= headers)


html_data= response.text
contenido=response.content
response.status_code


soup = BeautifulSoup(html_data, "html.parser")
soup2 = BeautifulSoup(contenido, "html.parser")
#print(soup)

tablas = soup.find_all("table")




#for index, table in enumerate(tablas):
    #print(table)
    #if ("Historical Revenue (Quarterly) Data" in str(table)):
        #print("entre")
        #indice = index
        #break

q_tesla = pd.DataFrame(columns=("Fecha", "Ingresos"))

for row in tablas[1].tbody.find_all("tr"):
    columnas= row.find_all("td")
    if columnas != []:
        Fecha= columnas[0].text
        Ingresos= columnas[1].text.replace("\n", " ").strip().replace("B", "00000000").replace(".", "")
        #Ingresos=' '.join(columnas[1].text.splitlines()).strip()
    
        temp_df = pd.DataFrame({"Fecha": [Fecha], "Ingresos": [Ingresos]})
        q_tesla = pd.concat([q_tesla, temp_df], ignore_index=True)
        
for row in tablas[0].tbody.find_all("tr"):
    columnas= row.find_all("td")
    if columnas != []:
        Fecha= columnas[0].text
        Ingresos= columnas[1].text.replace("\n", " ").strip().replace("B", "00000000").replace(".", "")
        #Ingresos=' '.join(columnas[1].text.splitlines()).strip()
    
        temp_df = pd.DataFrame({"Fecha": [Fecha], "Ingresos": [Ingresos]})
        q_tesla = pd.concat([q_tesla, temp_df], ignore_index=True)


q_tesla.head()



,Fecha,Ingresos
0,"March 31, 2018",340900000000
1,"December 31, 2017",328800000000
2,"September 30, 2017",298500000000
3,"June 30, 2017",279000000000
4,"March 31, 2017",269600000000
